# 2nd Notebook for multiple linear regression models

In [1]:
# Standard Packages
import pandas as pd
import numpy as np

# Viz Packages
import seaborn as sns
import matplotlib.pyplot as plt

# Scipy Stats
import scipy.stats as stats 

# Statsmodel Api
import statsmodels.api as sm
from statsmodels.formula.api import ols

# SKLearn Modules
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.feature_selection import f_regression

# Notebook Options
import warnings
warnings.filterwarnings("ignore", category= FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pd.options.display.max_columns = None
pd.options.display.width = None

In [9]:
# Formula to calculate regression results
def regression_results(y_test, y_pred):
    
    # Compute regression metrics
    explained_variance=metrics.explained_variance_score(y_test, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_test, y_pred) 
    mse=metrics.mean_squared_error(y_test, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_test, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_test, y_pred)
    r2=metrics.r2_score(y_test, y_pred)
    
    # Display formatted metrics
    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [2]:
# Load Baseline DF
mlr_baseline_df = pd.read_csv('../data/baseline_500k_to_15mil.csv')
mlr_baseline_df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,lat,long,renovated,zip,month,year
0,675000.0,4,1.0,1180,7140,1.0,0,0,0,0,4,7,Gas,PUBLIC,1180,0,0,40,1969,47.461975,-122.190520,0,98055,5,2022
1,920000.0,5,2.5,2770,6703,1.0,0,0,1,2,3,7,Oil,PUBLIC,1570,1570,0,240,1950,47.711525,-122.355910,0,98133,12,2021
2,775000.0,3,3.0,2160,1400,2.0,0,0,0,2,3,9,Gas,PUBLIC,1090,1070,200,270,2010,47.566110,-122.290200,0,98118,12,2021
3,592500.0,2,2.0,1120,758,2.0,0,0,1,0,3,7,Electricity,PUBLIC,1120,550,550,30,2012,47.532470,-122.071880,0,98027,8,2021
4,625000.0,2,1.0,1190,5688,1.0,0,0,1,0,3,7,Electricity,PUBLIC,1190,0,300,0,1948,47.763470,-122.340155,0,98133,7,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26116,719000.0,3,2.5,1270,1141,2.0,0,0,0,0,3,8,Gas,PUBLIC,1050,420,200,60,2007,47.690440,-122.370620,0,98117,10,2021
26117,1555000.0,5,2.0,1910,4000,1.5,0,0,0,0,4,8,Oil,PUBLIC,1600,1130,0,210,1921,47.664740,-122.329400,0,98103,11,2021
26118,1313000.0,3,2.0,2020,5800,2.0,0,0,0,1,3,7,Gas,PUBLIC,2020,0,0,520,2011,47.565610,-122.388510,0,98116,6,2021
26119,800000.0,3,2.0,1620,3600,1.0,0,0,1,0,3,7,Gas,PUBLIC,940,920,240,110,1995,47.610395,-122.295850,0,98122,5,2022


In [3]:
# Load school score by zip data
school_scores_df = pd.read_csv('../data/school_scores_by_zip.csv')
school_scores_df

,Unnamed: 0,ZIPCODE,percent_met_standard,district
0,0,98034,0.720261,LAKE WASHINGTON
1,1,98074,0.720261,LAKE WASHINGTON
2,2,98034,0.720261,LAKE WASHINGTON
3,3,98053,0.720261,LAKE WASHINGTON
4,4,98074,0.720261,LAKE WASHINGTON
...,...,...,...,...
649,649,98070,0.589803,VASHON ISLAND
650,650,98070,0.589803,VASHON ISLAND
651,651,98070,0.589803,VASHON ISLAND
652,652,98070,0.589803,VASHON ISLAND


In [4]:
# Drop extra index and district
school_scores_df = school_scores_df[['ZIPCODE', 'percent_met_standard']]
school_scores_df

,ZIPCODE,percent_met_standard
0,98034,0.720261
1,98074,0.720261
2,98034,0.720261
3,98053,0.720261
4,98074,0.720261
...,...,...
649,98070,0.589803
650,98070,0.589803
651,98070,0.589803
652,98070,0.589803


In [5]:
# rename zipcode column for merge with baseline df
school_scores_df['zip'] = school_scores_df['ZIPCODE']
school_scores_df = school_scores_df[['percent_met_standard','zip']]
school_scores_df

/var/folders/wf/s7j2jn9j1ns035j4rx9g3rzr0000gn/T/ipykernel_32761/2146252363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_scores_df['zip'] = school_scores_df['ZIPCODE']


,percent_met_standard,zip
0,0.720261,98034
1,0.720261,98074
2,0.720261,98034
3,0.720261,98053
4,0.720261,98074
...,...,...
649,0.589803,98070
650,0.589803,98070
651,0.589803,98070
652,0.589803,98070


In [7]:
# Drop duplicate zip codes 
school_scores_df.drop_duplicates(subset='zip', inplace=True)
school_scores_df

/var/folders/wf/s7j2jn9j1ns035j4rx9g3rzr0000gn/T/ipykernel_32761/2278327757.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_scores_df.drop_duplicates(subset='zip', inplace=True)


,percent_met_standard,zip
0,0.720261,98034
1,0.720261,98074
3,0.720261,98053
7,0.720261,98033
14,0.720261,98052
...,...,...
626,0.633615,98051
629,0.690543,98045
635,0.690543,98024
648,0.589803,98070


In [8]:
# Join scores table with baseline on zip
merged_df = mlr_baseline_df.join(school_scores_df.set_index('zip'), on='zip', how='inner')
merged_df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,lat,long,renovated,zip,month,year,percent_met_standard
0,675000.0,4,1.0,1180,7140,1.0,0,0,0,0,4,7,Gas,PUBLIC,1180,0,0,40,1969,47.461975,-122.190520,0,98055,5,2022,0.368621
36,750000.0,3,2.0,1830,7969,1.0,0,0,0,0,3,7,Gas,PUBLIC,930,930,240,90,1950,47.466730,-122.214000,1,98055,3,2022,0.368621
97,728000.0,4,2.0,2170,7520,1.0,0,0,0,0,3,7,Gas,PUBLIC,1240,1240,490,60,1973,47.463930,-122.189740,0,98055,3,2022,0.368621
200,565000.0,4,2.0,1400,10364,1.5,0,0,0,0,4,6,Electricity,PUBLIC,1400,0,330,330,1971,47.448450,-122.212430,0,98055,3,2022,0.368621
271,645000.0,3,2.0,1520,8250,1.0,0,0,0,0,3,8,Gas,PUBLIC,1190,590,420,200,1981,47.460870,-122.188690,0,98055,12,2021,0.368621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4120,950000.0,3,2.0,2600,64626,1.5,0,0,0,3,3,8,Gas,PRIVATE,2600,0,0,360,2009,47.718260,-121.405660,0,98288,10,2021,0.476190
8053,619000.0,4,1.0,1350,21640,1.5,1,0,0,2,4,5,Electricity,PRIVATE,1350,0,0,280,1964,47.694940,-121.298765,0,98288,8,2021,0.476190
12868,560000.0,2,2.0,1170,63217,1.5,0,0,0,0,3,8,Gas,PRIVATE,1170,0,0,190,2002,47.717600,-121.404800,0,98288,9,2021,0.476190
15196,869300.0,3,2.5,3610,44686,2.0,0,0,1,0,5,7,Gas,PUBLIC,2310,1300,0,440,1923,47.708820,-121.354160,0,98288,11,2021,0.476190


In [10]:
# Convert price to log price and drop price
merged_df['log_price'] = np.log1p(merged_df['price'])
merged_df = merged_df.drop(columns=['price'], axis=1)
merged_df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,lat,long,renovated,zip,month,year,percent_met_standard,log_price
0,4,1.0,1180,7140,1.0,0,0,0,0,4,7,Gas,PUBLIC,1180,0,0,40,1969,47.461975,-122.190520,0,98055,5,2022,0.368621,13.422469
36,3,2.0,1830,7969,1.0,0,0,0,0,3,7,Gas,PUBLIC,930,930,240,90,1950,47.466730,-122.214000,1,98055,3,2022,0.368621,13.527830
97,4,2.0,2170,7520,1.0,0,0,0,0,3,7,Gas,PUBLIC,1240,1240,490,60,1973,47.463930,-122.189740,0,98055,3,2022,0.368621,13.498058
200,4,2.0,1400,10364,1.5,0,0,0,0,4,6,Electricity,PUBLIC,1400,0,330,330,1971,47.448450,-122.212430,0,98055,3,2022,0.368621,13.244583
271,3,2.0,1520,8250,1.0,0,0,0,0,3,8,Gas,PUBLIC,1190,590,420,200,1981,47.460870,-122.188690,0,98055,12,2021,0.368621,13.377007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4120,3,2.0,2600,64626,1.5,0,0,0,3,3,8,Gas,PRIVATE,2600,0,0,360,2009,47.718260,-121.405660,0,98288,10,2021,0.476190,13.764218
8053,4,1.0,1350,21640,1.5,1,0,0,2,4,5,Electricity,PRIVATE,1350,0,0,280,1964,47.694940,-121.298765,0,98288,8,2021,0.476190,13.335862
12868,2,2.0,1170,63217,1.5,0,0,0,0,3,8,Gas,PRIVATE,1170,0,0,190,2002,47.717600,-121.404800,0,98288,9,2021,0.476190,13.235694
15196,3,2.5,3610,44686,2.0,0,0,1,0,5,7,Gas,PUBLIC,2310,1300,0,440,1923,47.708820,-121.354160,0,98288,11,2021,0.476190,13.675445


In [17]:
# Drop Sewer and one hot encode heat source
merged_df = merged_df.drop('sewer_system', axis=1)
df_heat_source = pd.get_dummies(merged_df['heat_source'], prefix='heat_source')
ed_mlr_df = pd.concat([merged_df, df_heat_source], axis=1)
ed_mlr_df.drop(columns='heat_source', axis=1, inplace=True)
ed_mlr_df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,lat,long,renovated,zip,month,year,percent_met_standard,log_price,heat_source_Electricity,heat_source_Gas,heat_source_Oil,heat_source_Solar_Other
0,4,1.0,1180,7140,1.0,0,0,0,0,4,7,1180,0,0,40,1969,47.461975,-122.190520,0,98055,5,2022,0.368621,13.422469,0,1,0,0
36,3,2.0,1830,7969,1.0,0,0,0,0,3,7,930,930,240,90,1950,47.466730,-122.214000,1,98055,3,2022,0.368621,13.527830,0,1,0,0
97,4,2.0,2170,7520,1.0,0,0,0,0,3,7,1240,1240,490,60,1973,47.463930,-122.189740,0,98055,3,2022,0.368621,13.498058,0,1,0,0
200,4,2.0,1400,10364,1.5,0,0,0,0,4,6,1400,0,330,330,1971,47.448450,-122.212430,0,98055,3,2022,0.368621,13.244583,1,0,0,0
271,3,2.0,1520,8250,1.0,0,0,0,0,3,8,1190,590,420,200,1981,47.460870,-122.188690,0,98055,12,2021,0.368621,13.377007,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4120,3,2.0,2600,64626,1.5,0,0,0,3,3,8,2600,0,0,360,2009,47.718260,-121.405660,0,98288,10,2021,0.476190,13.764218,0,1,0,0
8053,4,1.0,1350,21640,1.5,1,0,0,2,4,5,1350,0,0,280,1964,47.694940,-121.298765,0,98288,8,2021,0.476190,13.335862,1,0,0,0
12868,2,2.0,1170,63217,1.5,0,0,0,0,3,8,1170,0,0,190,2002,47.717600,-121.404800,0,98288,9,2021,0.476190,13.235694,0,1,0,0
15196,3,2.5,3610,44686,2.0,0,0,1,0,5,7,2310,1300,0,440,1923,47.708820,-121.354160,0,98288,11,2021,0.476190,13.675445,0,1,0,0


In [19]:
# Declare features and target variable
X = ed_mlr_df.drop('log_price', axis=1)
y = ed_mlr_df['log_price']

# Split the data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Create LR Model
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [20]:
regression_results(y_test, y_pred)

explained_variance:  0.7453
mean_squared_log_error:  0.0003
r2:  0.7453
MAE:  0.1797
MSE:  0.062
RMSE:  0.2489


In [ ]:
f_values, p_values = f_regression(X, y)
print(f_values, p_values)

In [ ]:
baseline_with_ed_summary = pd.DataFrame(lr.coef_, X.columns, columns=['Coefficients'])
print(all_num_feature_summary_df)

In [ ]:
# Create feature summary df
baseline_with_ed_summary['f_values'] = f_values
baseline_with_ed_summary['p_values'] = p_values
baseline_with_ed_summary

# Commuting MLR

In [30]:
# Use latlong to compute distance from each address to Amazon and University of Washington (wikipedia and geohack)

## Declare amazon coordinates
amzn_lat = 47.615144
amzn_long = -122.338578

msft_lat = 47.641944
msft_long = -122.127222

## Declare University of Washington coordinates
wash_lat = 47.654167
wash_long = -122.308056

In [31]:
# Duplicate DF first
### code here

# Define formulat to calculate the distances between the lat long coordinate sets
from math import sin, cos, sqrt, atan2
def haversine(lat1, lon1, lat2, lon2):

    # Calculates the Haversine distance between two points on the Earth's surface.
    R = 6371  # radius of Earth in km
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c
    return d

ed_mlr_df["distance_to_amazon"] = haversine(amzn_lat, amzn_long, ed_mlr_df["lat"], ed_mlr_df["long"])
ed_mlr_df["distance_to_msft"] = haversine(msft_lat, msft_long, ed_mlr_df["lat"], ed_mlr_df["long"])
ed_mlr_df["distance_to_wash"] = haversine(wash_lat, wash_long, ed_mlr_df["lat"], ed_mlr_df["long"])

ed_mlr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26075 entries, 0 to 25461
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bedrooms                 26075 non-null  int64  
 1   bathrooms                26075 non-null  float64
 2   sqft_living              26075 non-null  int64  
 3   sqft_lot                 26075 non-null  int64  
 4   floors                   26075 non-null  float64
 5   waterfront               26075 non-null  int64  
 6   greenbelt                26075 non-null  int64  
 7   nuisance                 26075 non-null  int64  
 8   view                     26075 non-null  int64  
 9   condition                26075 non-null  int64  
 10  grade                    26075 non-null  int64  
 11  sqft_above               26075 non-null  int64  
 12  sqft_basement            26075 non-null  int64  
 13  sqft_garage              26075 non-null  int64  
 14  sqft_patio            

In [32]:
ed_mlr_df.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,lat,long,renovated,zip,month,year,percent_met_standard,log_price,heat_source_Electricity,heat_source_Gas,heat_source_Oil,heat_source_Solar_Other,distance_to_amazon,distance_to_msft,distance_to_wash
0,4,1.0,1180,7140,1.0,0,0,0,0,4,7,1180,0,0,40,1969,47.461975,-122.19052,0,98055,5,2022,0.368621,13.422469,0,1,0,0,20.337222,20.567735,23.119222
36,3,2.0,1830,7969,1.0,0,0,0,0,3,7,930,930,240,90,1950,47.466730,-122.21400,1,98055,3,2022,0.368621,13.527830,0,1,0,0,18.968165,20.542453,22.004537
97,4,2.0,2170,7520,1.0,0,0,0,0,3,7,1240,1240,490,60,1973,47.463930,-122.18974,0,98055,3,2022,0.368621,13.498058,0,1,0,0,20.187751,20.342688,22.940951
200,4,2.0,1400,10364,1.5,0,0,0,0,4,6,1400,0,330,330,1971,47.448450,-122.21243,0,98055,3,2022,0.368621,13.244583,1,0,0,0,20.814923,22.445961,23.974044
271,3,2.0,1520,8250,1.0,0,0,0,0,3,8,1190,590,420,200,1981,47.460870,-122.18869,0,98055,12,2021,0.368621,13.377007,0,1,0,0,20.515316,20.656210,23.285361


In [33]:
# Declare features and target variable
X = ed_mlr_df.drop('log_price', axis=1)
y = ed_mlr_df['log_price']

# Split the data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Create LR Model
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [34]:
regression_results(y_test, y_pred)

explained_variance:  0.7792
mean_squared_log_error:  0.0002
r2:  0.7791
MAE:  0.1661
MSE:  0.0537
RMSE:  0.2318


In [35]:
f_values, p_values = f_regression(X, y)
print(f_values, p_values)

[3.32569728e+03 8.83228071e+03 1.90653883e+04 2.68024176e+02
 1.09231991e+03 7.06437795e+02 2.26651902e+02 6.92440591e-02
 2.65337961e+03 2.79696619e-01 1.68763909e+04 1.25714927e+04
 1.61477459e+03 2.13898045e+03 2.86497134e+03 1.49029822e+02
 4.58356802e+03 4.23741744e+02 2.51021144e+02 4.54887332e+02
 1.41001673e+02 2.35910734e+02 9.12176579e+03 5.68450083e+02
 7.86947118e+02 1.60013409e+02 2.84380538e+01 1.68030965e+03
 9.55064733e+03 2.98588045e+03] [0.00000000e+000 0.00000000e+000 0.00000000e+000 6.09346472e-060
 1.07413057e-234 1.32724943e-153 5.24920935e-051 7.92442704e-001
 0.00000000e+000 5.96904634e-001 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 3.50260845e-034
 0.00000000e+000 2.07122479e-093 2.84853347e-056 4.46723182e-100
 1.94911341e-032 5.23236043e-053 0.00000000e+000 2.61521685e-124
 1.27224628e-170 1.43956356e-036 9.75498815e-008 0.00000000e+000
 0.00000000e+000 0.00000000e+000]


In [36]:
ed_and_distance_summary = pd.DataFrame(lr.coef_, X.columns, columns=['Coefficients'])
print(ed_and_distance_summary)

                         Coefficients
bedrooms                -1.784800e-03
bathrooms                2.504322e-02
sqft_living              9.241295e-05
sqft_lot                 5.648050e-07
floors                  -4.848696e-02
waterfront               3.154355e-01
greenbelt                3.589392e-02
nuisance                -2.842012e-02
view                     6.259266e-02
condition                4.235418e-02
grade                    9.702846e-02
sqft_above               1.410458e-04
sqft_basement            2.260098e-05
sqft_garage              1.402698e-05
sqft_patio               4.637958e-05
yr_built                -8.559039e-04
lat                     -7.624774e-01
long                     2.103420e-01
renovated                7.946976e-02
zip                     -5.474306e-04
month                    8.570503e-03
year                     1.930045e-01
percent_met_standard     8.489576e-01
heat_source_Electricity -3.088492e-02
heat_source_Gas          2.871019e-04
heat_source_

In [37]:
# Create feature summary df
ed_and_distance_summary['f_values'] = f_values
ed_and_distance_summary['p_values'] = p_values
ed_and_distance_summary

,Coefficients,f_values,p_values
bedrooms,-1.784800e-03,3325.697277,0.000000e+00
bathrooms,2.504322e-02,8832.280709,0.000000e+00
sqft_living,9.241295e-05,19065.388294,0.000000e+00
sqft_lot,5.648050e-07,268.024176,6.093465e-60
floors,-4.848696e-02,1092.319912,1.074131e-234
waterfront,3.154355e-01,706.437795,1.327249e-153
greenbelt,3.589392e-02,226.651902,5.249209e-51
nuisance,-2.842012e-02,0.069244,7.924427e-01
view,6.259266e-02,2653.379612,0.000000e+00
condition,4.235418e-02,0.279697,5.969046e-01
